final project items
- descibe the data and a summary of the stats
- state the research question, and what exactly we're trying to express with the data
- what is the method? why this visual? why is this one chosen?
- what's next? what conclusion can be drawn? what comes next for analyzing the data?

selenium documentation (for reader reference): 
* https://selenium-python.readthedocs.io/
* https://www.selenium.dev/documentation/

proxy list (in case the current proxy ip can't be used): https://free-proxy-list.net/

In [10]:
#importing our libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import time
import csv
import datetime
import json

In [11]:
#booting up selenium to automate the scraping of reddit

#defining proxy ip
#if we get rate limited we must change the proxy
#more proxies can be accessed from the link above
proxy = "71.14.218.2"

#defining headless mode (if we want to run this operation without gui)
def driversetup(proxy):
    options= webdriver.FirefoxOptions()
    options.add_argument("-headless")
    options.add_argument(f"--proxy-server={proxy}")
    return webdriver.Firefox(options=options)
 
browser= driversetup(proxy)

#defining non-headless mode
#browser= webdriver.Firefox()

In [12]:
#helper function to open a link to a page
def openlink(url):
    browser.get(url) #open the url in browser (we will be using firefox)

In [13]:
#helper function to get title and body text of post
#also get info like time, link, etc which may be helpful

def getpostinfo(url, day, keyword):
    #opening the link
    openlink(url)
    time.sleep(5) #wait for page to load
    
    #get number representing the date
    data= {'word' : keyword}
    
    #locate the top bar where you can tab between json, raw data, and header
    topbar= browser.find_element(By.CSS_SELECTOR, "ul[class='tabs-menu']")
    rawtab= topbar.find_element(By.CSS_SELECTOR, "a[id='rawdata-tab']") #locating rawdata tab
    ActionChains(browser).click(rawtab).perform() #click on tab
    
    #identify the raw data tab
    rawdata= browser.find_element(By.CSS_SELECTOR, "div[id='rawdata-panel']")
    #get the data in text form
    posts= rawdata.find_element(By.CSS_SELECTOR, "pre[class='data']").text
    postdata= json.loads(posts) #convert the string (a dict) into a dict

    data.update(postdata) #add data from query into our dictionary
    
    return data

In [14]:

def getunix(year, month, day):
    day= datetime.datetime(year, month, day, 0, 0)
    return int(time.mktime(day.timetuple()))-21600


In [15]:
#reddit caps search results at 100, so instead we will separate our query by day and scrape the results
#every timestamp in unix must be subtracted by 21600 because we're in central time

#defining the day we want to start querying. 
#we will query every month from 1/1/2019 to 1/1/2025
firstdayend= getunix(2019, 1, 31)
firstdaystart= getunix(2018, 12, 31)

#adding helper function so we can easily reset the time each time we query
def resetfirstend():
    return firstdayend

def resetfirststart():
    return firstdayend

In [16]:
#before and after are the variables we use when querying urls
before= firstdayend
after= firstdaystart

#defining the end of our sampling to be 1/1/2025
lastdayend= datetime.datetime(2020, 1, 1, 0, 0)
end= getunix(2025, 1, 1)

In [17]:
'''#quick debugging test

before= 1704585600
after= 1704499200
end= 1735689600

postdata_unalive= [] #defining empty list for the data

url= f"https://api.pullpush.io/reddit/search/submission/?q=unalive&subreddit=offmychest&sort=asc&before={before}&after={after}"

scrapedata= getpostinfo(url, after, "unalive")
postdata_unalive.append(scrapedata)
print(postdata_unalive)'''

'#quick debugging test\n\nbefore= 1704585600\nafter= 1704499200\nend= 1735689600\n\npostdata_unalive= [] #defining empty list for the data\n\nurl= f"https://api.pullpush.io/reddit/search/submission/?q=unalive&subreddit=offmychest&sort=asc&before={before}&after={after}"\n\nscrapedata= getpostinfo(url, after, "unalive")\npostdata_unalive.append(scrapedata)\nprint(postdata_unalive)'

In [18]:
#scraping for 'unalive'
postdata_unalive= [] #defining empty list for the data

#exit and break do not do anything in jupyter notebook
while before<=end:
    url= f"https://api.pullpush.io/reddit/search/submission/?q=unalive&subreddit=offmychest&sort=asc&before={before}&after={after}"
    scrapedata= getpostinfo(url, after, "unalive")
    postdata_unalive.append(scrapedata)

    print(f"got data for {after} to {before}")
    
    after=before
    #if our usual increment will take us past our end year, query until end year
    #if we are at the end year, exit the loop
    if before==end:
        before=before+2592000
    elif (before+2592000)>end:
        before= end
    #otherwise query for the next 30 days
    else:
        before=before+2592000

#reset the start parameters  
before= firstdayend
after= firstdaystart

browser.quit()

got data for 1548892800 to 1546214400
got data for 1551484800 to 1548892800
got data for 1554076800 to 1551484800
got data for 1556668800 to 1554076800
got data for 1559260800 to 1556668800
got data for 1561852800 to 1559260800
got data for 1564444800 to 1561852800
got data for 1567036800 to 1564444800
got data for 1569628800 to 1567036800
got data for 1572220800 to 1569628800
got data for 1574812800 to 1572220800
got data for 1577404800 to 1574812800
got data for 1579996800 to 1577404800
got data for 1582588800 to 1579996800
got data for 1585180800 to 1582588800
got data for 1587772800 to 1585180800
got data for 1590364800 to 1587772800
got data for 1592956800 to 1590364800
got data for 1595548800 to 1592956800
got data for 1598140800 to 1595548800
got data for 1600732800 to 1598140800
got data for 1603324800 to 1600732800
got data for 1605916800 to 1603324800
got data for 1608508800 to 1605916800
got data for 1611100800 to 1608508800
got data for 1613692800 to 1611100800
got data for

In [19]:
#combine our data into a singular list
postdata= postdata_unalive

#after collecting data, export it into csv format
header = ["word", "date", "data", "metadata", "error"]
with open('unalive_data.csv', 'w', encoding="utf-8", newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(postdata)